# Changes to Particle Flow Network for Deep Sets

## Goals
(In Order)
 - [ ] Experiment with adding TNet to Inputs
 - [ ] Add dropout layers in latent space network
 - [ ] Add Attention Layers
 - [ ] Build Model which handles Tracks Separately

In [3]:
import numpy as np
import matplotlib.pyplot as plt
from matplotlib import cm
from matplotlib.colors import Normalize
from matplotlib.colors import LogNorm
import time as t
from time import perf_counter as cput
import scipy.constants as spc
import matplotlib.ticker as ticker

import sys
sys.path.append('/home/russbate/MLPionCollaboration/LCStudies/')
from util import resolution_util as ru
from util import plot_util as pu
from util import deep_set_util as dsu

## Declare GPUs and Import TensorFlow

In [2]:
import os
os.environ['CUDA_VISIBLE_DEVICES'] = "3"
os.environ['TF_FORCE_GPU_ALLOW_GROWTH'] = 'true'
# os.environ['TF_GPU_ALLOCATOR']='cuda_malloc_async'

import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers

## IMPORTANT ## ====== ## DISABLE EAGER EXECUTION WITH TensorFlow!! ##
from tensorflow.python.framework.ops import disable_eager_execution
disable_eager_execution()

print(tf.__version__)

2.6.0


## Load Data
### Data Including Tracks
### X = [Energy, Eta, Phi, rPerp, Track Flag, Layer]
### Y = [truthPartE, truthTrackPt, cluster_ENG_CALIB_TOT]

In [ ]:
t0 = t.time()
Xraw = np.load('/data/rbate/X_STMC_502_files.npy', mmap_mode='r')[:175000,:,:]
Yraw = np.load('/data/rbate/Y_STMC_502_files.npy', mmap_mode='r')[:175000,:]

X = np.lib.format.open_memmap('/data/rbate/X_STMC_notebook.npy',
                             mode='w+', dtype=np.float64, shape=(Xraw.shape[0], Xraw.shape[1], 5))
np.copyto(dst=X, src=Xraw[:,:,:5], casting='same_kind', where=True)

Y = np.lib.format.open_memmap('/data/rbate/Y_STMC_notebook.npy',
                             mode='w+', dtype=np.float64, shape=(Yraw.shape[0]))
np.copyto(dst=Y, src=Yraw[:,0], casting='same_kind', where=True)

t1 = t.time()
print('Time to load memory mapped data: '+str(t1-t0)+' (s)')

# Models

In [ ]:
def ParticleFlow_base(num_points, num_features, name="Russell Flow Network"):
    
    inputs = keras.Input(shape=(num_points, num_features), name='input')

    dense_0 = layers.Dense(100)
    t_dist_0 = layers.TimeDistributed(dense_0, name='t_dist_0')(inputs)
    activation_0 = layers.Activation('relu', name="activation_0")(t_dist_0)
    
    dense_1 = layers.Dense(100)
    t_dist_1 = layers.TimeDistributed(dense_1, name='t_dist_1')(activation_0)
    activation_1 = layers.Activation('relu', name='activation_1')(t_dist_1)
    
    dense_2 = layers.Dense(128)
    t_dist_2 = layers.TimeDistributed(dense_2, name='t_dist_2')(activation_1)
    activation_2 = layers.Activation('relu', name='activation_2')(t_dist_2)
    
    masking_layer = layers.Masking(mask_value=0.0)
    
    lambda_layer = layers.Lambda(point_mask_fn, output_shape=(None, None),
                                mask=None,
                                name='mask')(inputs)

    sum_layer = layers.Dot(axes=(1,1), name='sum')([lambda_layer, activation_2])
    
    dense_3 = layers.Dense(100, name='dense_0')(sum_layer)
    activation_3 = layers.Activation('relu', name="activation_3")(dense_3)
    
    dense_4 = layers.Dense(100, name='dense_1')(activation_3)
    activation_4 = layers.Activation('relu', name="activation_4")(dense_4)
    
    dense_5 = layers.Dense(100, name='dense_2')(activation_4)
    activation_5 = layers.Activation('relu', name="activation_5")(dense_5)
    
    dense_6 = layers.Dense(1, name='output')(activation_5)
    activation_6 = layers.Activation('relu', name="activation_6")(dense_6)
    
    return keras.Model(inputs=inputs, outputs=activation_6, name=name)

In [ ]:
muh_modle = ParticleFlow_base(num_points=100, num_features=4)
muh_modle.summary()